I. 데이터 분석 연습하기

In [35]:
#1. 데이터 탐색
#데이터 가져오기
import pandas as pd
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/y_train.csv', encoding='CP949')

#행/열 확인
# print(x_train.head())
# print(x_train.shape)
# print(x_test.head())
# print(x_test.shape)
# print(y_train.head())
# print(y_train.shape)

#요약정보 확인
# print(x_train.info())

#기초통계량 확인
# print(x_train.describe())

#2. 전처리
#불필요한 칼럼 삭제
x_test_cust_id = x_test['cust_id']
x_train = x_train.drop(columns= 'cust_id')
x_test = x_test.drop(columns= 'cust_id')
y_train = y_train.drop(columns= 'cust_id')

#결측치 처리
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())

x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)

# print(x_train.isnull().sum())
# print(x_test.isnull().sum())

#범주형 변수 인코딩 - 주구매상품, 주구매지점
# print(x_train.head())
# print(x_train['주구매상품'].unique())
# print(x_train['주구매지점'].unique())
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
# print(x_train['주구매상품'].head(10))
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])

x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
# print(x_train['주구매지점'].head(10))
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])

#파생변수 만들기 - 환불금액
# print(x_train.head(10))
condition = x_train['환불금액'] > 0
x_train.loc[condition, '환불금액_new'] = 1
x_train.loc[~condition, '환불금액_new'] = 0
# print(x_train[['환불금액', '환불금액_new']])
x_train = x_train.drop(columns= '환불금액')

condition = x_test['환불금액'] > 0
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0
# print(x_test[['환불금액', '환불금액_new']])
x_test = x_test.drop(columns= '환불금액')

#표준화 크기변환 - 기초통계량 표준화 크기 변환
# print(x_train.describe())
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns= x_test.columns)
# print(x_train.describe().T)

#상관관계 확인 - 총구매액, 최대구매액, 환불금액_new
# print(x_train.head())
# print(x_train[['총구매액', '최대구매액', '환불금액_new']].corr())
x_train = x_train.drop(columns= '최대구매액')
x_test = x_test.drop(columns= '최대구매액')

#3. 모델 학습 및 평가
#데이터 학습 & 하이퍼 파라미터 튜닝
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=10, criterion='entropy', random_state=10)
model.fit(x_train, y_train)
y_test_pred = model.predict(x_test)
# print(pd.DataFrame(y_test_pred).head())

#결과 예측
y_test_proba = model.predict_proba(x_test)
# print(pd.DataFrame(y_test_proba)[.head()])
# print(pd.DataFrame(y_test_proba)[1])
result = pd.DataFrame(y_test_proba)[1]

#모델 평가 - roc_auc_score
y_train_pred = model.predict(x_train)
from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_train, y_train_pred))

from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
# print(X_TRAIN.shape)
# print(X_TEST.shape)
# print(Y_TRAIN.shape)
# print(Y_TEST.shape)

model = DecisionTreeClassifier(max_depth=10, criterion='entropy', random_state=42)
model.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = model.predict(X_TEST)

# print(roc_auc_score(Y_TEST, Y_TEST_PRED))

#4. 결과 제출
#제출 코드
final = pd.concat([x_test_cust_id, result], axis=1).rename(columns= {1:'gender'})
final.to_csv('/content/drive/MyDrive/bigData-main/004001009.csv', index=False)

#정상 여부 확인
final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009.csv')
print(final)

      cust_id    gender
0        3500  0.800000
1        3501  0.000000
2        3502  0.000000
3        3503  0.451163
4        3504  0.451163
...       ...       ...
2477     5977  0.462312
2478     5978  0.451163
2479     5979  0.560000
2480     5980  0.000000
2481     5981  0.512821

[2482 rows x 2 columns]


In [70]:
# import math
# import time

# start = time.time()
# math.factorial(100000)
# end = time.time()

# print(f"{end - start:.5f} sec")

#0. 문제 의도 파악
#고객 3,500명의 상품 구매속성 통해 '남자일 확률' 예측하는 모형 만들기(성별 분류모델)
#gender가 result

#1. 데이터 탐색하기
#라이브러리, 데이터
import pandas as pd
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/y_train.csv', encoding='CP949')

#행/열 구조 확인: shape 숫자 주석으로 써두기
# print(x_train.shape) -> (3500, 10)
# print(x_test.shape) -> (2482, 10)
# print(y_train.shape) -> (3500, 2)
# print(x_train.head())
# print(x_test.head())
# print(y_train.head())
#cust_id: 불필요, 환불금액: 결측치 처리, 주구매상품&주구매지점: 라벨 인코딩, 총구매액&최대구매액&환불금액: 상관관계 파악

#요약정보
# print(x_train.info())

#기초통계량 확인: 데이터 스케일링 필요한지 여부 확인
# print(x_train.describe())

#2. 전처리
#불필요한 칼럼 삭제
x_train = x_train.drop(columns= 'cust_id')
y_train = y_train.drop(columns= 'cust_id')
x_test_cust_id = x_test['cust_id']
x_test = x_test.drop(columns= 'cust_id')

#결측치 처리
# print(x_train.isnull().sum())
# print(x_train['환불금액'].head(30))
x_train['환불금액'] = x_train['환불금액'].fillna(0)
# print(x_train.isnull().sum())

#범주형 변수 인코딩: 주구매상품, 주구매지점
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])
# print(x_train['주구매상품'].head())

x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])
# print(x_train['주구매지점'].head())

#파생변수 만들기: 환불금액
# print(x_train.head())
condition = x_train['환불금액'] > 0
x_train.loc[condition, '환불금액_new'] = 1
x_train.loc[~condition, '환불금액_new'] = 0
x_train = x_train.drop(columns= '환불금액')

condition = x_test['환불금액'] > 0
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] =0
x_test = x_test.drop(columns='환불금액')

# print(x_train.head())
# print(x_test.head())

#표준화 크기 변환
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns= x_test.columns)

#상관관계 확인
# print(x_train[['총구매액', '최대구매액', '환불금액_new']].corr())
x_train = x_train.drop(columns= '최대구매액')
x_test = x_test.drop(columns= '최대구매액')

#3. 모델 학습 및 평가: 의사결정나무
#학습용/검증용 데이터 분리
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
# print(X_TRAIN.shape) -> (2800, 8)
# print(X_TEST.shape) -> (700, 8)
# print(Y_TRAIN.shape) -> (2800, 1)
# print(Y_TEST.shape) -> (700, 1)

#데이터 학습 & 하이퍼 파라미터 튜닝
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=3, criterion='entropy', random_state=42)
model.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = model.predict(X_TEST)

#결과 예측
y_test_proba = model.predict_proba(x_test)
# print(pd.DataFrame(y_test_proba).head())
result = pd.DataFrame(y_test_proba)[1]

#모델 평가
from sklearn.metrics import roc_auc_score
y_train_pred = model.predict(x_train)
# print(roc_auc_score(y_train, y_train_pred))
# print(roc_auc_score(Y_TEST, Y_TEST_PRED))

#4. 결과 제출
#제출 코드
final = pd.concat([x_test_cust_id, result], axis=1).rename(columns= {1: 'gender'})
final.to_csv('/content/drive/MyDrive/bigData-main/004001009.csv', index= False)

#정상 여부 확인
final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009.csv')
print(final)

0.25745 sec
0.5834794637986127
0.5653335735302948
      cust_id    gender
0        3500  0.287834
1        3501  0.287834
2        3502  0.179445
3        3503  0.469038
4        3504  0.469038
...       ...       ...
2477     5977  0.531605
2478     5978  0.469038
2479     5979  0.308468
2480     5980  0.330153
2481     5981  0.531605

[2482 rows x 2 columns]


II. 분류 모델 수행하기

In [104]:
#0. 문제 의도 파악
#코드 실행 시간
import math
import time
start = time.time()
math.factorial(100000)
end = time.time()
print(f"{end - start:.5f} sec")

#의도 정리: 고객 데이터 통해 '생존 여부' 분류하는 모델 만들기

#1. 데이터 탐색
#라이브러리, 데이터 로드
import pandas as pd
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_x_train.csv', encoding= 'CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_x_test.csv', encoding= 'CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_y_train.csv', encoding= 'CP949')

#행/열 구조 파악
#PassengerId: 삭제했다가 마지막에 concat, 승객이름&티켓번호: 삭제, 성별: 수동 인코딩, 객실번호: 파생변수
# print(x_train.head())
# print(x_train.shape) -> (891, 11)
# print(x_test.head())
# print(x_test.shape) -> (418, 11)
# print(y_train.head())
# print(y_train.shape) -> (891, 2)

#요약정보
# print(x_train.info())

#기초통계량 -> 데이터 스케일링 필요(표준화 크기변환)
# print(x_train.describe())

#2. 전처리
#불필요한 칼럼 삭제: PassengerId, 승객이름, 티켓번호
x_test_PassengerId = x_test['PassengerId']

x_train = x_train.drop(columns= ['PassengerId', '승객이름', '티켓번호'])
x_test = x_test.drop(columns= ['PassengerId', '승객이름', '티켓번호'])
y_train = y_train.drop(columns= '癤풮assengerId')
# print(y_train)
# print(x_train.head())

#결측치 처리: 나이=평균값, 객실번호=삭제, 선착장=최빈값
# print(x_train.isnull().sum())
# print(x_train['나이'].head(30))
# print(x_train['객실번호'].head(30))
# print(x_train['선착장'].head(30))

x_train_age_mean = x_train['나이'].mean()
x_test_age_mean = x_test['나이'].mean()
x_train['나이'] = x_train['나이'].fillna(x_train_age_mean)
# print(x_train['나이'].head(30))
x_test['나이'] = x_test['나이'].fillna(x_test_age_mean)

x_train = x_train.drop(columns= ['객실번호'])
x_test = x_test.drop(columns= ['객실번호'])

선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
x_train = x_train.drop(columns= '선착장')
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns= '선착장')

#범주형 변수 인코딩: 성별(남자0, 여자1)
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
# print(x_train['성별'].head())
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

#파생변수 만들기: 가족수(형제자매배우자수+부모자식수)
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
# print(x_test[['가족수', '형제자매배우자수', '부모자식수']])
x_train = x_train.drop(columns= ['형제자매배우자수', '부모자식수'])
x_test = x_test.drop(columns= ['형제자매배우자수', '부모자식수'])
# print(x_train.head())

#표준화 크기 변환
# print(x_train.info())
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns= x_test.columns)
# print(x_train)

#상관관계 확인 -> 불필요한 칼럼 삭제
# print(x_train.corr())

#3. 모델 학습 및 평가
#학습용/검증용 데이터 분리
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)
# print(X_TRAIN.shape)
# print(X_TEST.shape)
# print(Y_TRAIN.shape)
# print(Y_TEST.shape)

#데이터 학습 & 하이퍼 파라미터 튜닝
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=42)
# print(help(XGBClassifier))
model.fit(X_TRAIN, Y_TRAIN)

#결과 예측
Y_TEST_PRED = pd.DataFrame(model.predict(X_TEST))
y_test_pred = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})

#모델 평가
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

#4. 결과 제출
#제출 코드: to_csv(index=False)
final = pd.concat([x_test_PassengerId, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009.csv', index=False)

#최종 확인: read_csv
# final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009.csv')
# final

0.27785 sec
0.7934813794686406


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,PassengerId,Survived
0,892.0,0
1,893.0,0
2,894.0,0
3,895.0,0
4,896.0,0
...,...,...
886,NaN,1
887,NaN,1
888,NaN,1
889,NaN,1


In [122]:
#0. 문제의도 파악
#소요시간 코드
import math
import time
start = time.time()
math.factorial(100000)
end = time.time()
# print(f"{end - start:.5f} sec")

#의도
#고객 데이터 토대로 생존여부 분류하는 모델

#1. 데이터 탐색
#라이브러리, 데이터 로드
import pandas as pd
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_y_train.csv', encoding='CP949')

#행/열 구조
# print(x_train.head())
# print(x_train.shape)
# print(x_test.head())
# print(x_test.shape)
# print(y_train.head())
# print(y_train.shape)

#요약정보 확인
# print(x_train.info())

#기초통계량 확인
# print(x_train.describe())

#PassengerId & 승객이름 & 티켓번호: 삭제, 성별: 수동인코딩, 형제자매배우자수&부모자식수: 파생변수(가족수)

#2. 전처리하기
#불필요한 칼럼 삭제
x_test_passengerid = x_test['PassengerId']

x_train = x_train.drop(columns= ['PassengerId', '승객이름', '티켓번호', '객실번호'])
x_test = x_test.drop(columns= ['PassengerId', '승객이름', '티켓번호', '객실번호'])
y_train = y_train.drop(columns= '癤풮assengerId')

#결측치 처리
# print(x_train.isnull().sum())
x_train_age_mean = x_train['나이'].mean()
x_train['나이'] = x_train['나이'].fillna(x_train_age_mean)
x_test_age_mean = x_test['나이'].mean()
x_test['나이'] = x_test['나이'].fillna(x_test_age_mean)

#범주형 변수 인코딩: 성별(남0 여1), 선착장
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns= {'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
x_train = x_train.drop(columns= '선착장')
# print(x_train.head())
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns= {'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns= '선착장')
# print(x_test.head())

#파생변수 만들기: 가족수
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_train = x_train.drop(columns=['형제자매배우자수', '부모자식수'])
x_test = x_test.drop(columns=['형제자매배우자수', '부모자식수'])
# print(x_train.head())

#표준화 크기 변환
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
# print(x_train.head())

#상관관계 확인
# print(x_train.corr())

#3. 모델 학습 및 평가
#학습용/검증용 데이터 분리
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)
# print(X_TRAIN.shape)
# print(X_TEST.shape)
# print(Y_TRAIN.shape)
# print(Y_TEST.shape)

#데이터 학습 & 하이퍼 파라미터
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=42)
model.fit(X_TRAIN, Y_TRAIN)

#결과 예측
Y_TEST_PRED = pd.DataFrame(model.predict(X_TEST)).rename(columns= {0: 'Survived'})
y_test_pred = pd.DataFrame(model.predict(x_test)).rename(columns= {0: 'Survived'})

#모델 평가
from sklearn.metrics import roc_auc_score
# print(roc_auc_score(Y_TEST, Y_TEST_PRED))

#4. 결과 제출
#제출 코드
final = pd.concat([x_test_passengerid, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009.csv', index=False)

#최종 확인
# final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009.csv')
# print(final)

0.25624 sec
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


III. 예측 모델 수행하기

In [131]:
#0. 문제의도 파악
#코드 소요시간 체크
import math
import time
start = time.time()
math.factorial(100000)
end = time.time()
print(f"{end - start:.5f} sec")

#의도
#고객의 자전거 대여 속성 통해 '시간별 자전거 대여량' 예측 모형 만들기

#1. 데이터 탐색
#라이브러리, 데이터 로드
import pandas as pd
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/bike_x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/bike_x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/bike_y_train.csv', encoding='CP949')

#행/열 구조
# print(x_train.head())
# print(x_train.shape)
# print(x_test.head())
# print(x_test.shape)
# print(y_train.head())
# print(y_train.shape)

#요약정보
# print(x_train.info())

#기초통계량: 데이터 스케일링 필요
# print(x_train.describe())

#datetime: 삭제, 공휴일&근무일: 둘중 하나 삭제?, 온도&체감온도: 파생변수

#2. 전처리
#불필요한 칼럼 삭제: datetime
x_test_datetime = x_test['datetime']
x_train = x_train.drop(columns= 'datetime')
x_test = x_test.drop(columns= 'datetime')
y_train = y_train.drop(columns= '癤풼atetime')

#결측치 처리
# print(x_train.isnull().sum())

#범주형 변수 인코딩

#파생변수 만들기

#표준화 크기변환

#상관관계 확인
print(x_train.corr())
x_train = x_train.drop(columns= '체감온도')
x_test = x_test.drop(columns= '체감온도')
print(x_train.head())

#3. 모델 학습 및 평가
#학습용/검증용 데이터 분리
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)
print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

#데이터 학습 & 하이퍼 파라미터 튜닝: XGBRegressor
from xgboost import XGBRegressor
model = XGBRegressor(max_depth=10, n_estimators=100, random_state=42)
model.fit(x_train, y_train)
# print(help(XGBRegressor))

#결과 예측
y_test_pred = pd.DataFrame(model.predict(x_test)).rename(columns= {0: 'count'})
print(y_test_pred)

#모델 평가

#4. 결과 제출
#제출 코드

#최종 확인

0.54873 sec
            계절       공휴일       근무일        날씨        온도      체감온도        습도  \
계절    1.000000  0.029368 -0.008126  0.008879  0.258689  0.264744  0.190610   
공휴일   0.029368  1.000000 -0.250491 -0.007074  0.000295 -0.005215  0.001929   
근무일  -0.008126 -0.250491  1.000000  0.033772  0.029966  0.024660 -0.010880   
날씨    0.008879 -0.007074  0.033772  1.000000 -0.055035 -0.055376  0.406244   
온도    0.258689  0.000295  0.029966 -0.055035  1.000000  0.984948 -0.064949   
체감온도  0.264744 -0.005215  0.024660 -0.055376  0.984948  1.000000 -0.043536   
습도    0.190610  0.001929 -0.010880  0.406244 -0.064949 -0.043536  1.000000   
풍속   -0.147121  0.008409  0.013373  0.007261 -0.017852 -0.057473 -0.318607   

            풍속  
계절   -0.147121  
공휴일   0.008409  
근무일   0.013373  
날씨    0.007261  
온도   -0.017852  
체감온도 -0.057473  
습도   -0.318607  
풍속    1.000000  
   계절  공휴일  근무일  날씨    온도  습도   풍속
0   1    0    0   1  9.84  81  0.0
1   1    0    0   1  9.02  80  0.0
2   1    0    0   1  9.02  